In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import api_key

# Configure gmaps
gmaps.configure(api_key=api_key)

In [2]:
weather_csv_file = "weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,0,Ushuaia,20,AR,1669580786,47,-54.8000,-68.3000,56.82,4.61
1,1,Hamilton,100,US,1669580532,79,39.1834,-84.5333,54.37,23.02
2,2,Mahébourg,20,MU,1669580789,83,-20.4081,57.7000,71.89,1.14
3,3,Shenzhen,78,CN,1669580734,97,22.5455,114.0683,77.85,5.75
4,4,Presidencia Roque Sáenz Peña,0,AR,1669580791,18,-26.7852,-60.4388,100.60,7.76


In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=api_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)


In [4]:
# Create a humidity Heatmap layer
heatmap = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(weather_df["Humidity"]), point_radius = 2)

heatmap.add_layer(heat_layer)

heatmap

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temperature"] < 80) & (filtered_weather_df["Max Temperature"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
57,57,Porbandar,0,IN,1669580864,44,21.6422,69.6093,73.44,8.68
152,152,Kapaa,0,US,1669580997,82,22.0752,-159.3190,76.98,4.61
180,180,Uthal,0,PK,1669581037,60,25.8072,66.6219,71.85,2.55
203,203,General Cepeda,0,MX,1669581071,11,25.3833,-101.4500,78.06,8.16
229,229,Upington,0,ZA,1669581107,19,-28.4478,21.2561,79.18,4.61
309,309,Nouakchott,0,MR,1669580919,53,18.0858,-15.9785,78.73,9.71
340,340,Makkah al Mukarramah,0,SA,1669581260,67,21.4267,39.8261,77.27,3.76
408,408,Makakilo City,0,US,1669581354,60,21.3469,-158.0858,78.69,9.22
435,435,Salalah,0,OM,1669581393,78,17.0151,54.0924,75.29,3.44
467,467,Matamoros,0,MX,1669581421,15,25.5333,-103.2500,76.10,0.00


In [6]:
# create hotel_df with hotel name column
hotel_df = filtered_weather_df
hotel_df["Hotel Name"] = np.nan
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
57,57,Porbandar,0,IN,1669580864,44,21.6422,69.6093,73.44,8.68,NaN
152,152,Kapaa,0,US,1669580997,82,22.0752,-159.3190,76.98,4.61,NaN
180,180,Uthal,0,PK,1669581037,60,25.8072,66.6219,71.85,2.55,NaN
203,203,General Cepeda,0,MX,1669581071,11,25.3833,-101.4500,78.06,8.16,NaN
229,229,Upington,0,ZA,1669581107,19,-28.4478,21.2561,79.18,4.61,NaN
309,309,Nouakchott,0,MR,1669580919,53,18.0858,-15.9785,78.73,9.71,NaN
340,340,Makkah al Mukarramah,0,SA,1669581260,67,21.4267,39.8261,77.27,3.76,NaN
408,408,Makakilo City,0,US,1669581354,60,21.3469,-158.0858,78.69,9.22,NaN
435,435,Salalah,0,OM,1669581393,78,17.0151,54.0924,75.29,3.44,NaN
467,467,Matamoros,0,MX,1669581421,15,25.5333,-103.2500,76.10,0.00,NaN


In [9]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories": 'accommodation.hotel',
    "apiKey":api_key,
    "limit": 20
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
   # print(name_address)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
   # print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
       # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Porbandar - nearest hotel: No hotel found
Kapaa - nearest hotel: No hotel found
Uthal - nearest hotel: No hotel found
General Cepeda - nearest hotel: No hotel found
Upington - nearest hotel: No hotel found
Nouakchott - nearest hotel: No hotel found
Makkah al Mukarramah - nearest hotel: No hotel found
Makakilo City - nearest hotel: No hotel found
Salalah - nearest hotel: No hotel found
Matamoros - nearest hotel: No hotel found


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Hotel Name
57,57,Porbandar,0,IN,1669580864,44,21.6422,69.6093,73.44,8.68,No hotel found
152,152,Kapaa,0,US,1669580997,82,22.0752,-159.3190,76.98,4.61,No hotel found
180,180,Uthal,0,PK,1669581037,60,25.8072,66.6219,71.85,2.55,No hotel found
203,203,General Cepeda,0,MX,1669581071,11,25.3833,-101.4500,78.06,8.16,No hotel found
229,229,Upington,0,ZA,1669581107,19,-28.4478,21.2561,79.18,4.61,No hotel found
309,309,Nouakchott,0,MR,1669580919,53,18.0858,-15.9785,78.73,9.71,No hotel found
340,340,Makkah al Mukarramah,0,SA,1669581260,67,21.4267,39.8261,77.27,3.76,No hotel found
408,408,Makakilo City,0,US,1669581354,60,21.3469,-158.0858,78.69,9.22,No hotel found
435,435,Salalah,0,OM,1669581393,78,17.0151,54.0924,75.29,3.44,No hotel found
467,467,Matamoros,0,MX,1669581421,15,25.5333,-103.2500,76.10,0.00,No hotel found


In [11]:
# Configure the map plot
final_plot = hotel_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
final_plot

AttributeError: 'DataFrame' object has no attribute 'hvplot'